In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
cwd = os.getcwd()  
df = pd.read_csv(cwd+'/shot_logs.csv')


In [6]:
df.columns

Index(['GAME_ID', 'MATCHUP', 'LOCATION', 'W', 'FINAL_MARGIN', 'SHOT_NUMBER', 'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME', 'SHOT_DIST', 'PTS_TYPE', 'SHOT_RESULT', 'CLOSEST_DEFENDER', 'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST', 'FGM', 'PTS', 'player_name', 'player_id'], dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 21 columns):
GAME_ID                       128069 non-null int64
MATCHUP                       128069 non-null object
LOCATION                      128069 non-null object
W                             128069 non-null object
FINAL_MARGIN                  128069 non-null int64
SHOT_NUMBER                   128069 non-null int64
PERIOD                        128069 non-null int64
GAME_CLOCK                    128069 non-null object
SHOT_CLOCK                    122502 non-null float64
DRIBBLES                      128069 non-null int64
TOUCH_TIME                    128069 non-null float64
SHOT_DIST                     128069 non-null float64
PTS_TYPE                      128069 non-null int64
SHOT_RESULT                   128069 non-null object
CLOSEST_DEFENDER              128069 non-null object
CLOSEST_DEFENDER_PLAYER_ID    128069 non-null int64
CLOSE_DEF_DIST                128069 non-null

In [8]:
df.describe()

,GAME_ID,FINAL_MARGIN,SHOT_NUMBER,PERIOD,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_id
count,1.280690e+05,128069.000000,128069.000000,128069.000000,122502.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000
mean,2.140045e+07,0.208723,6.506899,2.469427,12.453344,2.023355,2.765901,13.571504,2.264670,159038.487284,4.123015,0.452139,0.997314,157238.251247
std,2.578773e+02,13.233267,4.713260,1.139919,5.763265,3.477760,3.043682,8.888964,0.441159,78791.172947,2.756446,0.497706,1.130978,79362.389336
min,2.140000e+07,-53.000000,1.000000,1.000000,0.000000,0.000000,-163.600000,0.000000,2.000000,708.000000,0.000000,0.000000,0.000000,708.000000
25%,2.140023e+07,-8.000000,3.000000,1.000000,8.200000,0.000000,0.900000,4.700000,2.000000,101249.000000,2.300000,0.000000,0.000000,101162.000000
50%,2.140045e+07,1.000000,5.000000,2.000000,12.300000,1.000000,1.600000,13.700000,2.000000,201949.000000,3.700000,0.000000,0.000000,201939.000000
75%,2.140067e+07,9.000000,9.000000,3.000000,16.675000,2.000000,3.700000,22.500000,3.000000,203079.000000,5.300000,1.000000,2.000000,202704.000000
max,2.140091e+07,53.000000,38.000000,7.000000,24.000000,32.000000,24.900000,47.200000,3.000000,530027.000000,53.200000,1.000000,3.000000,204060.000000


### first only looking at one specific player to test hot hand hypothesis

In [9]:
#checking to see if any other player has last name conley
mc_df = df[df['player_name'] == 'mike conley']
c_df = df[df['player_name'].str.contains('conley')]
mc_df.equals(c_df)

True

In [98]:
def hh_alg(test_pattern,index):
    make = 0
    miss = 0
    output_dict = {
        'pattern': ''.join(map(str,test_pattern)),
        'len': len(test_pattern),
        #define hot hand as making a shot after 2 consecutive makes
        'hh' : 0,
        'nhh' : 0,
        #counting pairs of 1s and 0s but must not use unrealized condition sets
        'c_hh' : 0,
        'c_nhh' : 0
    }
    
    #count of makes and miss
    for i,r in enumerate(test_pattern):
        if r == 1:
            #checking for pairs of 1s but excluding unrealized condition sets
            if i >= 1:
                if r == test_pattern[i-1] and i != (len(test_pattern)-1):
                    output_dict['c_hh'] += 1
            #if a 1 occurs after a pair of 1s
            if make >= 2:
                output_dict['hh'] += 1
            #if a 0 occurs after a pair of 1s
            elif miss >= 2:
                output_dict['nhh'] += 1
            miss = 0
            make += 1
        else:
            #checking for pairs of 0s
            if i >= 1:
                if r == test_pattern[i-1] and i != (len(test_pattern)-1):
                    output_dict['c_nhh'] += 1
            make = 0
            miss += 1

    df = pd.DataFrame(output_dict, index=[index])
    return df

In [99]:
games = mc_df['GAME_ID'].unique()

In [100]:
for i,game in enumerate(games):
    test_game = mc_df[mc_df['GAME_ID'] == game]
    test_pattern = np.array(test_game['FGM'])
    if i == 0:
        df = hh_alg(test_pattern,i)
    else:
        df2 = hh_alg(test_pattern,i)
        df = df.append(df2)

In [101]:
df

,pattern,len,hh,nhh,c_hh,c_nhh
0,101101110,9,1,0,3,0
1,01100100010,11,0,2,1,3
2,1010010000101,13,0,2,0,4
3,00100001000,11,0,2,0,5
4,00001011000001,14,0,2,1,7
5,0110010110,10,0,1,2,1
6,101101010000100,15,0,1,1,3
7,0100100,7,0,1,0,1
8,01000000,8,0,0,0,4
9,010110101110,12,1,0,3,0


In [13]:
test_pattern

array([1, 0, 1, 1, 0, 1, 1, 1, 0])

In [53]:
test_pattern = np.array([1,1,1,0,1,0,0,1,1,0,0,0,0,0,1,1])

In [54]:
len(test_pattern)

16

In [95]:
hh_alg(test_pattern,10)

{'pattern': '01100101101010', 'len': 14, 'hh': 0, 'nhh': 0, 'c_hh': 0, 'c_nhh': 0}


,pattern,len,hh,nhh,c_hh,c_nhh
10,01100101101010,14,0,1,2,1
